In [23]:
# Устанавливаем бибилотеки
!pip install ultralytics roboflow --quiet

In [24]:
#импортируем необходимые библотеки
import os
from roboflow import Roboflow
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os
from google.colab import files
import shutil

In [25]:
#датасет для обучения
rf = Roboflow(api_key="sfzfWQgbVAN4uhzSfFAs")
project = rf.workspace("capston-invasive-plants-dataset").project("invasive-plants")
dataset = project.version(3).download("yolov8")
print(f"\nДатасет скачан: {dataset.location}")

loading Roboflow workspace...
loading Roboflow project...

Датасет скачан: /content/Invasive-Plants-3


In [26]:
# Обучаем модель
model = YOLO('yolov8n.pt')

train_results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=15,
    imgsz=640,
    batch=16,
    name='invasive_detector',
    project='/content/runs',
    exist_ok=True,
    save_dir='/content/runs')

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Invasive-Plants-3/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=invasive_detector, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.

In [32]:
# Подготавливаем файл для скачивания
model_src = "/content/runs/invasive_detector/weights/best.pt"
model_dst = "/content/invasive_detector_best.pt"

# Копируем
shutil.copy(model_src, model_dst)

# скачиваем на компьютер
files.download(model_dst)
print(f'Модель invasive_detector_best скачана на компьютер в папку для загрузок по умолчанию')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Модель invasive_detector_best скачана на компьютер в папку для загрузок по умолчанию


In [33]:
#скачиваем датасет2
rf = Roboflow(api_key="sfzfWQgbVAN4uhzSfFAs")
project2 = rf.workspace("test-yjlpb").project("hogweed-dataset")
dataset2 = project2.version(9).download("yolov8")
print(f"\nДатасет скачан: {dataset2.location}")

loading Roboflow workspace...
loading Roboflow project...

Датасет скачан: /content/Hogweed-Dataset-9


In [34]:
# Загружаем и дообучаем модель
#выбрать файл с моделью и подгрузить в сессионное хранилище Колаб
uploaded = files.upload()
uploaded_filename = list(uploaded.keys())[0]
model_path = f"/content/{uploaded_filename}"

Saving invasive_detector_best.pt to invasive_detector_best (1).pt


In [35]:
model = YOLO(model_path)

results = model.train(
    data=f"{dataset2.location}/data.yaml",
    epochs=15,
    imgsz=640,
    batch=16,
    patience=5,
    exist_ok=True,
    name='invasive_detector_hogweed',
    project='/content/runs',
    save_dir='/content/runs')


New https://pypi.org/project/ultralytics/8.3.236 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Hogweed-Dataset-9/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/invasive_detector_best (1).pt, momentum=0.937, mosaic=1.0, multi_scale=False, name

In [36]:
# Подготавливаем файл для скачивания
model_src2 = "/content/runs/invasive_detector_hogweed/weights/best.pt"
model_dst2 = "/content/hogweed_detector_best.pt"

# Копируем
shutil.copy(model_src2, model_dst2)

# скачиваем на компьютер
files.download(model_dst2)
print(f'Модель hogweed_detector_best скачана на компьютер в папку для загрузок по умолчанию')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Модель hogweed_detector_best скачана на компьютер в папку для загрузок по умолчанию


In [38]:
#оценка модели invasive_detector_best

#загружаем дообученную модель
uploaded = files.upload() #выбрать из папки модель hogweed_detector_best
uploaded_filename = list(uploaded.keys())[0]
model_path = f"/content/{uploaded_filename}"
model = YOLO(model_path)

#проверяем параметры модели
metrics = model.val()
Precision=metrics.box.p[0]
Recall=metrics.box.r[0]
mAP50=metrics.box.map50
print(f'\nТочность (Precision): {Precision:.2%} - % правильных обнаружений')
print(f'Полнота (Recall): {Recall:.2%} - % найденных растений')
print(f'Общая оценка (mAP50): {mAP50:.2%}')
f1 = 2 * (Precision * Recall) / (Precision + Recall)
print(f'\nF1-Score: {f1:.3f}')

Saving invasive_detector_best.pt to invasive_detector_best (2).pt
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.1±0.0 ms, read: 362.3±69.9 MB/s, size: 91.6 KB)
val: Scanning /content/Invasive-Plants-3/valid/labels.cache... 411 images, 50 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 411/411 514.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 26/26 3.6s/it 1:34
                   all        411        377      0.749      0.728      0.776      0.433
      Creeping Thistle        103        104      0.735      0.788      0.817      0.416
      Himalayan Balsam        114        117      0.758      0.821      0.826      0.382
     Japanese Knotweed         28         28      0.734      0.714      0.748      0.572
          Leafy Spurge         59         69      0.712    

In [39]:
#оценка модели invasive_detector_best

#загружаем дообученную модель
uploaded = files.upload() #выбрать из папки модель hogweed_detector_best
uploaded_filename = list(uploaded.keys())[0]
model_path = f"/content/{uploaded_filename}"
model = YOLO(model_path)

#проверяем параметры модели
metrics = model.val()
Precision=metrics.box.p[0]
Recall=metrics.box.r[0]
mAP50=metrics.box.map50
print(f'\nТочность (Precision): {Precision:.2%} - % правильных обнаружений')
print(f'Полнота (Recall): {Recall:.2%} - % найденных растений')
print(f'Общая оценка (mAP50): {mAP50:.2%}')
f1 = 2 * (Precision * Recall) / (Precision + Recall)
print(f'\nF1-Score: {f1:.3f}')

Saving hogweed_detector_best.pt to hogweed_detector_best (4).pt
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1669.9±640.2 MB/s, size: 83.3 KB)
val: Scanning /content/Hogweed-Dataset-9/valid/labels.cache... 93 images, 29 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 93/93 132.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 3.6s/it 21.8s
                   all         93        184      0.374      0.328      0.255      0.121
Speed: 2.6ms preprocess, 212.1ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to /content/runs/detect/val5

Точность (Precision): 37.39% - % правильных обнаружений
Полнота (Recall): 32.78% - % найденных растений
Общая оценка (mAP50): 25.51%

F1-Score: 0.349
